# Probabilistic Methods (Discrete)

<hr style="border:2px solid gray">

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

import warnings
warnings.filterwarnings("ignore")

When you have a variable that is not too noisy, you can have a baseline probability, then as new datapoints come in you can identify and compare the actual rate with the expected rate (probability).

<hr style="border:2px solid gray">

# Acquire the data

<div class="alert alert-block alert-info">
<b>Note:</b> 
Ada, was Codeup's first Data Science cohort! The data comes from the webserver logs of the API that we used in the timeseries module. Each row is one request to the server.
</div>

In [ ]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [ ]:
#get url connection
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
#read in from sql
df = pd.read_sql('SELECT * FROM api_access', url)
#parse data using function above
df = df.entry.apply(parse_log_entry)
#take a look
df.head()

<b> For the sake of this lesson we will create some sythetic records and append them to our existing data frame.

In [ ]:
#create new dataframe with fake data
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

#add the new dataframe we created to the original Ada information
new_df = df.append(new)

In [ ]:
new_df.sample(5)

<hr style="border:2px solid gray">

# Detecting Anomalies in Discrete Variables
<b>Anomaly Detection</b>: Identification of items, events or observations which is significantly different from the remaining data.

To detect anomalies in descrete variables we can look at <u>two</u> ways of summarizing the variable:
- <b>Count</b>: the number of times each unique value appears in the dataset
    - (ex): How many time a value of variable (e.g. ip address) shows up?
- <b>Frequencies</b>: the number of times each unique value appears in the dataset as a percentage of the total; the count divided by the total number of observations
    - <u>More</u> frequently occuring - <i>less</i> likely to be an anomaly
    - <u>Less</u> frequently occuring - <i>more</i> likely to be an anomaly

<hr style="border:1px solid gray">

<b>Examples of each type of Anomaly Detection technique</b>:
- How many times we see an ip address in the dataset
    - count how many times that ip address logs in
- What is probability of ip address showing up in the dataset
    - ip count / total observations
- Conditional probability. Given an ip address, what is prob of a particular status
    - probability of A, given that B is true
    - e.g authentication failure

### Let's take a look at ```ip``` using the count and frequency methods 

In [ ]:
#a count of each ip address
new_df.ip.value_counts().head()

In [ ]:
#what percent is that of the dataset
new_df.ip.value_counts(normalize=True).head()

In [ ]:
#turn this into a dataframe
ip_df = pd.DataFrame(new_df.ip.value_counts(dropna=False)).reset_index().\
                rename(columns={'index': 'ip', 'ip': 'count'})

ip_df

<b>We can define a function that combines both these pieces of information:</b>

In [ ]:
def value_counts_and_frequencies(s: pd.Series, dropna=True) -> pd.DataFrame:
    return pd.merge(
        s.value_counts(dropna=False).rename('count'),
        s.value_counts(dropna=False, normalize=True).rename('proba'),
        left_index=True,
        right_index=True,
    )

In [ ]:
#create a dataframe of actual count and percentage of occurances
ip_df = value_counts_and_frequencies(new_df.ip)
ip_df.head()

In [ ]:
#Let's visualize this
ip_df['count'].sort_values().tail(5).plot.barh(figsize=(5, 9))
None

<div class="alert alert-block alert-success">
<b>Takeaways:</b>
    
It looks like <u>one</u> ip address is accounting for almost 86% of the hits to the codeup curriculum.
</div>

<hr style="border:1px solid gray">

### Conditional Probabilities: Probabilities using 2 discrete variables

<b>prob(A|B) = prob(A&B)/prob(B)</b>

If we are looking for an unexpected status (like authentication failure) from a known/common IP address.

<div class="alert alert-block alert-info">
<b>HTTP Status Codes</b>

- 200: ok
- 3xx: redirects
- 4xx: client level errors -- the requester did something wrong
- 5xx: server level errors -- the server did something wrong

</div>

<b>What is probability of a certain status code given an IP address?

In [ ]:
#look at ip address, their status and the probability it will be that status
status_given_ip = (
    new_df.groupby('ip')
    .status_code.value_counts(normalize=True)
    .rename('proba_status_given_ip')
    .reset_index()
)
status_given_ip.head()

In [ ]:
#what about statuses that are NOT successful?
status_given_ip[status_given_ip.status_code != '200']

<b>This is interesting -- we have detected two ip addresses who only received 301 response codes. Let's investigate one further:

In [ ]:
#what about ips that had different statuses
status_given_ip[status_given_ip.proba_status_given_ip < 1]

<b>This is interesting -- 72.181.113.170 was successful on most attempts but not all. Same applies to 97.105.19.58

### Let's investigate Further

In [ ]:
#why is this ip address only returning 301 errors??
new_df[new_df.ip =='97.105.15.120']

<div class="alert alert-block alert-success">
<b>Takeaways:</b>

If the probability of a status given the ip address is 100%, we know that that was the only status code seen by that ip address. To detect anomalies here we could look for two things:
- Cases where the probability is < 100% -- this means that multiple different status codes were encountered from the given ip address
<br>

    <b>OR</b>
<br>

- Status codes other than 200. 200 is the most common response code in our dataset by far.
                                       
</div>

<hr style="border:2px solid gray">

## Using the original ADA dataframe (with no new entries)

In [ ]:
#make sure the original has 4 less entries
len(df), len(new_df)

In [ ]:
#look at ip address, their status and the probability it will be that status
og_status_given_ip = (
    df.groupby('ip')
    .status_code.value_counts(normalize=True)
    .rename('proba_status_given_ip')
    .reset_index()
)
og_status_given_ip.sample(10)

In [ ]:
# Essentially a list of the probability of each ip/status code combination
og_status_given_ip

In [ ]:
#set date to index, merge on ip and status
df = df.reset_index().merge(status_given_ip, on=['ip', 'status_code'], how='left').fillna(value=0).set_index('timestamp')

In [ ]:
#only give me the ips that have less than 15% unsuccessful accesses
df[df.proba_status_given_ip < 0.15]

<b>All low probability combinations are now easily identifiable with all additional data intact.

<b>All of these IPs have low probability of failure